In [ ]:
import matplotlib.pyplot as plt
import numpy as np

In [ ]:
import tensorflow as tf

from sklearn.model_selection import train_test_split
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
import os

In [ ]:
DATA_FILE_PATH= os.path.join(os.path.dirname(os.getcwd()),r"DataGathering\Data\30_30")
#print(DATA_FILE_PATH)

dir_list = os.listdir(DATA_FILE_PATH)
max_data_points = len(dir_list)
print("Number of data points: {}".format(len(dir_list)))
print(dir_list[0])

#print(os.listdir(os.path.join(DATA_FILE_PATH,dir_list[0])))

In [ ]:
def getAgent(path):

    filesToPull = os.listdir(path)

    dataPoints = []

    for fileName in filesToPull:
        try:

            data = np.load(os.path.join(path,fileName),allow_pickle=True)

            xPhys = data['a']
            forces = data['b']
            degreesOfFreedom = data['c']
            formating_array = data['d']

            maxDof = forces.shape[0]
            toPad = maxDof - len(degreesOfFreedom)
            degreesOfFreedom = np.pad(degreesOfFreedom,[0,toPad])

            compliance = formating_array[0]
            compliance_max = formating_array[1]
            shape = formating_array[2:]

            #forces = np.reshape(forces,(shape[0],shape[1],4))

            dataPoints.append([xPhys,forces,degreesOfFreedom,compliance_max,shape,compliance])

        except:
            print("Error in reading file couldn't get file: {}".format(fileName))
    
    return dataPoints




In [ ]:
#get some basig information about the shapes of the data
d1 = getAgent(os.path.join(DATA_FILE_PATH,dir_list[0]))

print("xPhys.shape:",d1[0][0].shape)
print("forces.shape:",d1[0][1].shape)
print("dofs.shape:",d1[0][2].shape)
print("c-Max:",d1[0][3])
print("compliance:",d1[0][5])
print("shape:",d1[0][4])

shape_of_Inputs = d1[0][4]

In [ ]:
d = d1[0][2]
print(d.shape)

d2 = np.ones(2*31*31)
for index in d:
    d2[index] = 0

d3 = np.reshape(d2,(31,31,2))
print(d3.shape)

d4 = d3.sum(2)
print(d4.shape)



In [ ]:
def formatData(xPhys,forces,degreesOfFreedom,compliance_max,shape):
    finalShape = (int(shape[0]+1),int(shape[1]+1))
    xPhysShape = xPhys.shape
    difference = (int(finalShape[0]-xPhysShape[0]),int(finalShape[1]-xPhysShape[1]))
    xPhys2 = np.pad(xPhys,[[0,difference[0]],[0,difference[1]]])

    forces2 = forces.sum(1)
    forces2 = np.reshape(forces2,(finalShape[0],finalShape[1],2))

    d2 = np.ones(2*finalShape[0]*finalShape[1])
    for index in degreesOfFreedom:
        d2[index] = 0
    d3 = np.reshape(d2,(finalShape[0],finalShape[1],2))
    degreesOfFreedom2 = d3.sum(2)

    c_max = compliance_max * np.ones((finalShape[0],finalShape[1]))

    arraysToStack = [xPhys2,forces2[:,:,0],forces2[:,:,1],degreesOfFreedom2]
    #for ar in arraysToStack:
    #    print(ar.shape)

    fullPart = np.stack(arraysToStack,axis=-1).astype('float32')

    return [xPhys2,forces2,degreesOfFreedom2,compliance_max]

    

In [ ]:
for xPhys,forces,degreesOfFreedom,compliance_max,shape,compliance in getAgent(os.path.join(DATA_FILE_PATH,dir_list[0])):
    data = formatData(xPhys,forces,degreesOfFreedom,compliance_max,shape)
    print(data[0].shape)
    print(data[1].shape)

In [ ]:
#Get the data
points_to_Use = 100000
"""
1000    = 11 seconds
10000   = 4 minutes
"""


points_to_Use = min(points_to_Use,max_data_points)

# can use an np.arrange and np.random.shuffle to get random data points every time
pointsShuffeled = np.arange(max_data_points).astype("int32")
np.random.shuffle(pointsShuffeled)


compliancesToMutateFrom = 0 # number of extra compliance_max value to add to add variety to the data
ComplianceRange = 10

bad


data_x = []
data_y = []
for i in range(points_to_Use):
    data1 = getAgent(os.path.join(DATA_FILE_PATH,dir_list[pointsShuffeled[i]]))
    for xPhys,forces,degreesOfFreedom,compliance_max,shape,compliance in data1: 
        x = formatData(xPhys,forces,degreesOfFreedom,compliance_max,shape)
        data_x.append(x)
        y_val = np.log(compliance).astype('float32')
        data_y.append(y_val)
        if(y_val <= 0):
            #print(i,compliance)
            data_x.pop()
            data_y.pop()
        for j in range(compliancesToMutateFrom):
            newC_max = compliance + (np.random.random()-0.5) * ComplianceRange
            x = formatData(xPhys,forces,degreesOfFreedom,compliance_max,shape)
            data_x.append(x)
            data_y.append(np.array([1*(newC_max >= compliance)]).astype('float32')) 

#data_x = np.array(data_x)
data_y = np.array(data_y)

print(len(data_x))
print(len(data_y))

In [ ]:
#get some statistics on the data
numTrue = 0
numFalse = 0

numTrue = int(data_y.sum())
numFalse = len(data_y) - numTrue

print(numTrue)
print(numFalse)

In [ ]:
#get some statistics on the data

yMax = np.max(data_y)
yMin = np.min(data_y)

print(yMax)
print(yMin)

In [ ]:
X_train, X_test, Y_train, Y_test  = train_test_split(data_x,data_y, test_size=0.25, random_state=42)
print("X_train: {}\nY_train: {}".format(len(X_train), len(Y_train)))
print("X_test: {}\nY_test: {}".format(len(X_test), len(Y_test)))

In [ ]:
X_train_part = []
X_train_compliance = []
X_train_forces=[]
X_train_supports = []

X_test_part = []
X_test_compliance = []
X_test_forces=[]
X_test_supports = []

for x,f,dof,compliance_max in X_train:
    X_train_part.append(x)
    X_train_forces.append(f)
    X_train_supports.append(dof)
    X_train_compliance.append(compliance)

for x,f,dof,compliance_max in X_test:
    X_test_part.append(x)
    X_test_forces.append(f)
    X_test_supports.append(dof)
    X_test_compliance.append(compliance)

X_train_part = np.array(X_train_part)
X_train_forces = np.array(X_train_forces)
X_train_supports = np.array(X_train_supports)
X_train_compliance = np.array(X_train_compliance)

X_test_part = np.array(X_test_part)
X_test_forces = np.array(X_test_forces)
X_test_supports = np.array(X_test_supports)
X_test_compliance = np.array(X_test_compliance)


In [ ]:
ToSavePath = os.path.join(os.path.dirname(os.getcwd()),r"Model Creation\models\complianceRegressor2")

cp_callback = tf.keras.callbacks.ModelCheckpoint(filepath=str(ToSavePath),
                                                     save_weights_only=True,
                                                     verbose=1)

InputShape = (31,31,4)
activation = 'relu'
uniformRandomInitalizer = tf.random_uniform_initializer(minval=-0.1, maxval=0.5)

the model will have a generic input of a 31 by 31 image with 4 channels representing the density grid, the forces in both the x and y direction, and the supports. A 5th channel, holding a matrix of compliance max values, will be stacked on the layers as they pass through each convolution. The same value will be concatenated in each dense layer once we flatten it.

In [ ]:
def m1():
    partInput = keras.Input(shape=InputShape,name="partImage")
    complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    x1 = layers.Conv2D(filters= 16,kernel_size=(3,3))(partInput)
    cScaling = layers.Reshape((31,31,1))(layers.RepeatVector(31*31)(complianceMaxInput))

    x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x2 = layers.MaxPooling2D(pool_size=(2,2))(x1)

    x2 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x3 = layers.MaxPooling2D(pool_size=(2,2))(x2)

    x_flat = layers.Flatten()(x3)
    x4 = layers.concatenate([x_flat,complianceMaxInput])
    x4 = layers.Dense(31,activation=activation)(x4)

    x4 = layers.concatenate([x4,complianceMaxInput])
    outputs = layers.Dense(2)(x4)
    return keras.Model(inputs= [partInput,complianceMaxInput],outputs=outputs)

In [ ]:
def m2():
    partInput = keras.Input(shape=InputShape,name="partImage")
    complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    x1 = layers.Conv2D(filters= 16,kernel_size=(3,3))(partInput)
    cScaling = layers.Reshape((31,31,1))(layers.RepeatVector(31*31)(complianceMaxInput))

    x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x1)

    newShape = (x1.shape[1],x1.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    x2 = layers.Concatenate()([x1,cScaling])
    x2 = layers.Conv2D(filters= 25, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.MaxPooling2D(pool_size=(2,2))(x2)

    x_flat = layers.Flatten()(x2)
    x3 = layers.concatenate([x_flat,complianceMaxInput])
    x3 = layers.Dense(31,activation=activation)(x3)

    x4 = layers.concatenate([x3,complianceMaxInput])
    outputs = layers.Dense(2)(x4)
    return keras.Model(inputs= [partInput,complianceMaxInput],outputs=outputs)

In [ ]:
def m3():
    partInput = keras.Input(shape=(31,31,1),name="partImage")
    forceInput = keras.Input(shape=(31,31,2),name="Forces")
    supportInput = keras.Input(shape=(31,31,1),name="Supports")
    complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    partConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(partInput)
    forceConv = layers.Conv2D(filters= 10,kernel_size=(3,3),padding='same',activation='tanh')(forceInput)
    supportConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(supportInput)
    cScaling = layers.Reshape((supportConv.shape[1],supportConv.shape[2],1))(layers.RepeatVector(supportConv.shape[1]*supportConv.shape[2])(complianceMaxInput))


    concatenatedConvolution = layers.Concatenate()([partConv,forceConv,supportConv,cScaling])
    x1 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(concatenatedConvolution)
    x1 = layers.Dropout(0.1)(x1)
    x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.Dropout(0.2)(x1)
    x1 = layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same')(x1)

    newShape = (x1.shape[1],x1.shape[2])
    #print(x1.shape)
    cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    #partScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(concatenatedConvolution)
    x2 = layers.Concatenate()([x1,cScaling])
    x2 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Dropout(0.3)(x2)
    x2 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x2)

    newShape = (x2.shape[1],x2.shape[2])
    #print(x1.shape)
    cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    x3 = layers.Concatenate()([x2,cScaling])
    x3 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x3)
    x3 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(x3)
    x3 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x3)

    x_flat = layers.Flatten()(x3)
    #x3 = layers.concatenate([x_flat,complianceMaxInput])
    x4 = layers.Dense(32,activation=activation)(x_flat)
    #x3 = layers.Dropout(0.2)(x3)

    #x4 = layers.concatenate([x3,complianceMaxInput])
    #outputs = layers.GlobalAveragePooling1D()(x4)
    outputs = layers.Dense(1)(x4)
    return keras.Model(inputs= [partInput,forceInput,supportInput,complianceMaxInput],outputs=outputs)


In [ ]:
def m4_regression():
    partInput = keras.Input(shape=(31,31,1),name="partImage")
    forceInput = keras.Input(shape=(31,31,2),name="Forces")
    supportInput = keras.Input(shape=(31,31,1),name="Supports")
    #complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    partConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(partInput)
    forceConv = layers.Conv2D(filters= 10,kernel_size=(3,3),padding='same',activation='tanh')(forceInput)
    supportConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(supportInput)
    #cScaling = layers.Reshape((supportConv.shape[1],supportConv.shape[2],1))(layers.RepeatVector(supportConv.shape[1]*supportConv.shape[2])(complianceMaxInput))


    concatenatedConvolution = layers.Concatenate()([partConv,forceConv,supportConv])
    x1 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(concatenatedConvolution)
    x1 = layers.Dropout(0.1)(x1)
    #x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.Dropout(0.2)(x1)
    x1 = layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same')(x1)

    newShape = (x1.shape[1],x1.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    partScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(concatenatedConvolution)
    x2 = layers.Concatenate()([x1,partScaling])
    x2 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Conv2D(filters= 128, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Dropout(0.3)(x2)
    x2 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x2)

    newShape = (x2.shape[1],x2.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    #x3 = layers.Concatenate()([x2,cScaling])
    x3 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x3 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(x3)
    x3 = layers.Dropout(0.3)(x3)
    x3 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x3)

    x_flat = layers.Flatten()(x3)
    #x3 = layers.concatenate([x_flat,complianceMaxInput])
    x4 = layers.Dense(32,activation=activation)(x_flat)
    #x3 = layers.Dropout(0.2)(x3)

    #x4 = layers.concatenate([x3,complianceMaxInput])
    #outputs = layers.GlobalAveragePooling1D()(x4)
    outputs = layers.Dense(1)(x4)
    return keras.Model(inputs= [partInput,forceInput,supportInput],outputs=outputs)


In [ ]:
def m5_regression():
    partInput = keras.Input(shape=(31,31,1),name="partImage")
    forceInput = keras.Input(shape=(31,31,2),name="Forces")
    supportInput = keras.Input(shape=(31,31,1),name="Supports")
    #complianceMaxInput = keras.Input(shape=(1),name="complianceMax")


    partConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(partInput)
    forceConv = layers.Conv2D(filters= 10,kernel_size=(3,3),padding='same',activation='tanh')(forceInput)
    supportConv = layers.Conv2D(filters= 5,kernel_size=(3,3),padding='same')(supportInput)
    #cScaling = layers.Reshape((supportConv.shape[1],supportConv.shape[2],1))(layers.RepeatVector(supportConv.shape[1]*supportConv.shape[2])(complianceMaxInput))


    concatenatedConvolution = layers.Concatenate()([partConv,forceConv,supportConv])
    x1 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(concatenatedConvolution)
    x1 = layers.Dropout(0.3)(x1)
    #x1 = layers.Concatenate()([x1,cScaling])
    x1 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x1)
    x1 = layers.GaussianNoise(stddev=0.5)(x1)
    x1 = layers.MaxPool2D(pool_size=(2,2),strides=(2,2),padding='same')(x1)

    newShape = (x1.shape[1],x1.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    partScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(concatenatedConvolution)
    x2 = layers.Concatenate()([x1,partScaling])
    x2 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.Dropout(0.3)(x2)
    x2 = layers.Conv2D(filters= 64, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x2 = layers.GaussianNoise(stddev=0.5)(x2)
    x2 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x2)

    newShape = (x2.shape[1],x2.shape[2])
    #print(x1.shape)
    #cScaling = layers.Resizing(height=newShape[0],width=newShape[1],interpolation='nearest')(cScaling)
    #x3 = layers.Concatenate()([x2,cScaling])
    x3 = layers.Conv2D(filters= 32, kernel_size=(3,3),padding='same',activation=activation)(x2)
    x3 = layers.Dropout(0.3)(x3)
    x3 = layers.Conv2D(filters= 16, kernel_size=(3,3),padding='same',activation=activation)(x3)
    x3 = layers.GaussianNoise(stddev=0.5)(x3)
    x3 = layers.MaxPooling2D(pool_size=(2,2),strides=(2,2),padding='same')(x3)

    x_flat = layers.Flatten()(x3)
    #x3 = layers.concatenate([x_flat,complianceMaxInput])
    x4 = layers.Dense(32,activation=activation)(x_flat)
    #x3 = layers.Dropout(0.2)(x3)

    #x4 = layers.concatenate([x3,complianceMaxInput])
    #outputs = layers.GlobalAveragePooling1D()(x4)
    outputs = layers.Dense(1)(x4)
    return keras.Model(inputs= [partInput,forceInput,supportInput],outputs=outputs)


In [ ]:
comp_tester = m5_regression()
comp_tester.summary()
#keras.utils.plot_model(comp_tester, os.path.join(os.getcwd(),"model1.png"), show_shapes=True)

In [ ]:
selfMadOptomizer = tf.keras.optimizers.Adam(learning_rate=0.001)
comp_tester.compile(optimizer=selfMadOptomizer, loss=tf.keras.losses.MeanSquaredError())
comp_tester.load_weights(str(ToSavePath))
# comp_tester.compile(optimizer=selfMadOptomizer,
#               loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
#               metrics=['accuracy'])

In [ ]:
comp_tester.fit(
                {"partImage":X_train_part, "Forces":X_train_forces, "Supports":X_train_supports},#, "complianceMax":X_train_compliance},
                Y_train,
                epochs=5,
                shuffle=True,
                validation_data=(
                    {"partImage":X_test_part, "Forces":X_test_forces, "Supports":X_test_supports},#,"complianceMax":X_test_compliance}, 
                        Y_test),
                callbacks=[cp_callback])

In [ ]:
pointsToTest = 10
dataSample_part = X_test_part[pointsToTest:]
dataSample_forces = X_test_forces[pointsToTest:]
dataSample_supports = X_test_supports[pointsToTest:]

dataSample_Y_True = Y_test[pointsToTest:]

dataSample_Y_pred = comp_tester.predict({"partImage":dataSample_part, "Forces":dataSample_forces, "Supports":dataSample_supports})


In [ ]:
meanError = 0
errorList = []

for i in range(pointsToTest):
    #print(dataSample_Y_True[i],dataSample_Y_pred[i][0])
    a = np.exp(dataSample_Y_True[i])
    b = np.exp(dataSample_Y_pred[i][0])
    if(a != 0):
        errorList.append(abs(a-b)/a)

error = np.array(errorList)

print("mean:", np.mean(error))
print("standard deviation:", np.std(error))
print("min: {}\tmax: {}".format(np.min(error),np.max(error)))
    